# How to Get to Your Signals - "don't be afraid to do simple things"
<img src="pictures/UP1_21032024_XG.png" alt="one day recording" width="600"/>

### Application of Simple STA/LTA Triggers 
<b> Developed by Tobias Megies; modified by J. Wassermann </b>

The goals is to see the effect of parameter changes on the selected/triggered waveforms. 
We start with the import of the relevant python/obspy modules/functions <br>
You also my consult the obspy tutorial on how to use and adjust trigger/picker (https://docs.obspy.org/tutorial/code_snippets/trigger_tutorial.html)

In [ ]:
import os
import subprocess
import optparse

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#core utilities
from obspy.core import UTCDateTime, Stream, AttribDict
from obspy import read_inventory
#SDS based data input
from obspy.clients.filesystem import sds

SDS_DATA_PATH = "/Users/jowa/Skience2026_Data/data_sds/"


The data set we are working with is from an experiment in 2024 at the lower part of the Grenzgletscher (glacier) 
located in the Mt. Rosa massif in Switzerland. <br>
In next code cell you can change the values of the triggers (see obspy documentation) as well as filter parameter etc.


<img src="pictures/Grenzgletscher_unten.jpg" alt="Grenz24 2500m" width="500"/> <img src="pictures/Lower_MH.jpg" alt="Gorner 2500m" width="500"/> 

We start by applying a simple recursive STA/LTA trigger ... but we use the network as a whole <br>
So we import the obspy function coincidence_trigger first

In [ ]:
#trigger
from obspy.signal.trigger import coincidence_trigger
par = AttribDict()

par.sds_rootdir = SDS_DATA_PATH # link to the data
#filter 
par.filter = AttribDict(type="bandpass", freqmin=5.0, freqmax=30.0,
                            corners=2, zerophase=True)
#different data sets from two different areas at the glacier
trace_ids = {}
#trace_ids = {"XG.UP1..GLZ": 1, "XG.UP2..GLZ": 1,"XG.UP3..GLZ": 1,"XG.UP4..GLZ": 1,"XG.UP5..GLZ": 1,
#            "XG.UP6..GLZ": 1}
trace_ids = {"XG.BB01..HHZ": 1, "XG.ADR1..GLZ": 1,"XG.C16..GLZ": 1,"XG.C17..GLZ": 1,"XG.C29..DLZ": 1,
             "XG.C2A..DLZ": 1,"XG.C68..DLZ": 1,"XG.C67..DLZ": 1,"XG.S35..GLZ": 1,"XG.C65..DLZ": 1,"XG.BAS..GLZ": 1,\
             "XG.C6A..DLZ": 1,"XG.69..DLZ": 1}
coinc_sum = 4.0 # at least for stations must trigger +- simultaneously
par.trace_ids = trace_ids

# STA/LTA settings - please note: length of sta and lta in seconds 
par.coincidence = AttribDict(trigger_type="recstalta", sta=1., lta=10,
                                 thr_on=2., thr_off=1.9,
                                 thr_coincidence_sum=coinc_sum,
                                 trace_ids=trace_ids, max_trigger_length=10,
                                 trigger_off_extension=1)
# output directory
par.dir = "./XG_trigger" 
par.logfile = os.path.join(par.dir, "%s_log.txt" % "XG")
par.trigfile = os.path.join(par.dir, "%s_trigger.txt" % "XG")

In [ ]:
# delete the old files to have a fresh start
!rm -f "%s/*.txt"%par.dir
!rm -rf "%s/*.png"%par.dir 

Now we do the triggering in hourly data chunks ...<br>
Pretty long loop ... and sorry for the warnings ... <br>
<b> Be carefull about the time intervall ... might be take along time </b>

In [ ]:
#select one day of data
start = UTCDateTime(2024,3,20)
end = UTCDateTime(2024,3,21)
t1 = int(UTCDateTime(start).timestamp)
t2 = int(UTCDateTime(end).timestamp)
times = [UTCDateTime(t) for t in np.arange(t1, t2, 3600)] #split the trigger in 1 hour pieces

sds_cl = sds.Client(sds_root=par.sds_rootdir)

for time in times:
    t1 = time
    t2 = t1 + 3600 + 10 #adding extra 10 seconds
    st = Stream()
    num_stations = 0
    station_id = []
    possible_coinc_sum = 0
    exceptions = []
    for trace_id, weight in par.trace_ids.items():
        net, sta, loc, cha = trace_id.split(".")
        station_id.append("%s.%s.%s.%s*"%(net,sta,loc,cha[:-1])) 
        for comp in "ZNE":
            cha_ = cha[:-1] + comp
            try:
               # load in the data channel by channel
                tmp = sds_cl.get_waveforms(network=net, station=sta, location=loc, channel=cha_, starttime=t1, endtime=t2)
                #read the metadata of the stations
                inv = read_inventory("./stationxml/station_%s_%s.xml"%(net,sta))
                tmp.attach_response(inv)
            except:
                exceptions.append("%s-%s" % (sta,cha_))
                continue
            if comp == cha[-1]:
                possible_coinc_sum += weight
            st += tmp
        num_stations += 1
    st.merge(-1)
    st.sort()

    #preparing the output log files and trigger
    trigger = []
    summary = []
    summary.append("#" * 79)
    summary.append("######## %s  ---  %s ########" % (t1, t2))
    summary.append("#" * 79)
    summary.append(st.__str__(extended=True))
    if exceptions:
        summary.append("#" * 33 + " Exceptions  " + "#" * 33)
        summary += exceptions
    summary.append("#" * 79)

    st.traces = [tr for tr in st if tr.stats.npts > 1]

    trig = []
    mutt = []
    if st:
        # preprocessing, backup original data for plotting at end
        st.detrend("linear")
        st.merge(method=1, fill_value=0)
        for tr in st:
            perc = 1.0 / (tr.stats.endtime - tr.stats.starttime)
            perc = min(perc, 1)
            tr.taper(type="cosine", max_percentage=perc)
            #we live in counts
            #tr.remove_sensitivity()

        #st.remove_response(water_level=10,output="VEL")
        st.sort()
        st.trim(t1, t2, pad=True, fill_value=0)
        st_trigger = st.copy()
        
        # filter and trigger
        st_trigger.filter(**par.filter)
############################### here we finally trigger ##################################
        # do the triggering (with additional data at sides to avoid artifacts
        trig = coincidence_trigger(stream=st_trigger, details=True,
                                  **par.coincidence)
        
        # restrict trigger list to time span of interest
        trig = [t for t in trig if (t1 <= t['time'] <= t2)]

############################### now we do the plotting ##################################
        for t in trig:
            max_similarity = max(list(t['similarity'].values()) + [0])
            time_str = str(t['time']).split(".")[0]
            sta_string = "-".join(t['stations'])
            info = "%s %ss %s %.2f %s"
            info = info % (time_str, ("%.1f" % t['duration']).rjust(4),
                    ("%i" % t['cft_peak_wmean']).rjust(3),
                    max_similarity, sta_string)
            summary.append(info)
            sta_string = ",".join(station_id)
            
            info = "%s %s %s %s"%\
                        (net, str(t['time']), str(t['duration']),str(t['coincidence_sum']))
            summary.append(info)
            trigger.append(info)
            tmp = st_trigger.slice(t['time'] - 1, t['time'] + t['duration'] + 1)
            filename = "%s_%.1f_%i_%s-%s_%.2f_%s.png"
            filename = filename % (time_str, t['duration'],
                                   t['cft_peak_wmean'], t['coincidence_sum'],
                                   possible_coinc_sum, max_similarity,
                                   net)

            #now we create figures for later checking of your trigger settings
            filename = os.path.join(par.dir, filename)
            stations = sorted(set([tr.id.rsplit(".", 1)[0] for tr in tmp]))
            dpi = 72
            fig = plt.figure(figsize=(700.0 / dpi, 400.0 / dpi))
            ax = None
            for i_, netstaloc in enumerate(stations):
                net, sta, loc = netstaloc.split(".")
                ax = fig.add_subplot(len(stations), 1, i_ + 1, sharex=ax)
                for comp, color in zip("ENZ", "rbk"):
                    tmp_ = tmp.select(network=net,station=sta,location=loc, component=comp).copy()
                    tmp_.detrend("constant")
                    for tr in tmp_:
                        x = tr.times() + (tr.stats.starttime - t['time'])
                        ax.plot(x, tr.data, color=color, linewidth=1.2)
                ax.text(0.02, 0.95, netstaloc, va="top", ha="left",
                        transform=ax.transAxes)
            for ax in fig.axes:
                ylims = ax.get_ylim()
                ax.set_yticks(ylims)
                #ax.set_yticklabels(["%.1e" % (val * 1) for val in ylims])
                ax.set_ylabel("a.u.")
            for ax in fig.axes[::2]:
                ax.yaxis.set_ticks_position("left")
            for ax in fig.axes[1::2]:
                ax.yaxis.set_ticks_position("right")
            for ax in fig.axes[:-1]:
                ax.set_xticks([])
            try:
                fig.tight_layout()
            except:
                pass
            fig.subplots_adjust(hspace=0)
            fig.savefig(filename, dpi=dpi)
            plt.close('all')

        del tmp
        del st_trigger
        del tr
    del st

    summary.append("#" * 79)
    summary = "\n".join(summary)
    trigger = "\n".join(trigger)
    # avoid writing long list of streams when using many event templates
    par_tmp = par.copy()
    summary += "\n" + "\n".join(("%s=%s" % (k, v) for k, v in par_tmp.items()))
    with open(par.logfile, "at") as fh:
        fh.write(summary + "\n")
    with open(par.trigfile, "at") as fu:
        fu.write(trigger + "\n")
    del summary
    del trigger
    del mutt

So let's have a look ....
Well still a lot of false triggers (of course depending what you expect)
Can we do better?
### Of Course if we use

# Application of an AR-Picker Schema

Pretty sofisticated picker which is using ar-prediction in combination with forward-backward STA/LTA. If tuned a very good picker also for S-waves....
### But ... the tuning is tedious 

Again lets load the needed libraries

In [ ]:
from obspy.core.event import Catalog, Event, Origin, Magnitude,  Arrival, Pick, WaveformStreamID
from obspy.core import UTCDateTime, Stream, AttribDict
from obspy import read_inventory
#from obspy.clients.fdsn import Client
from obspy.clients.filesystem import sds
#import operator
#import matplotlib

#from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
#from matplotlib.transforms import offset_copy

import os
#import optparse
import warnings
from copy import deepcopy
from configparser import ConfigParser, NoOptionError, NoSectionError

from obspy.signal.trigger import ar_pick

#just ment as an example how to use external cfg files .... but here for simplicity we set the ar parameter right in the code
#config_file = "./ar_picker.cfg"
#config = ConfigParser(allow_no_value=True)
# make all config keys case sensitive
#config.optionxform = str
#config.read(config_file)

For further insights of the functionality please refer to the ObsPy documentation (http://docs.obspy.org)

In [ ]:
def _arpicker(stream):
    """
    Run AR picker on all streams and set P/S picks accordingly.
    Also displays a message.
    """
    try:
        f1 = 5.0 #config.getfloat("ar_picker", "f1")
        f2 = 50.0 #config.getfloat("ar_picker", "f2")
        sta_p = 0.05 #config.getfloat("ar_picker", "sta_p")
        lta_p = 0.5 #0.5 #config.getfloat("ar_picker", "lta_p")
        sta_s = 0.05 #config.getfloat("ar_picker", "sta_s")
        lta_s = 0.5 #0.5 #config.getfloat("ar_picker", "lta_s")
        m_p = 2 #config.getint("ar_picker", "m_p")
        m_s = 4 #config.getint("ar_picker", "m_s")
        l_p = 0.1 #config.getfloat("ar_picker", "l_p")
        l_s = 0.1 #config.getfloat("ar_picker", "l_s")
    except (NoOptionError, NoSectionError) as e:
        msg = ('To use AR Picker, you need to have to set the following'
               'keys set: "f1", '
               '"f2", "lta_p", "sta_p", "lta_s", "sta_s", "m_p", "m_s", '
               '"l_p", "l_s" (compare documentation for '
               'obspy.signal.trigger.ar_pick\n%s') % str(e)
        self.error(msg)
        return

    try:
        z = stream.select(component="Z")[0]
        n = stream.select(component="N")[0]
        e = stream.select(component="E")[0]
    except IndexError:
        msg = ('AR picker currently only implemented for Z/N/E data, '
               'but provided stream was:\n%s') % stream
        print(msg)

    try:
        assert z.stats.sampling_rate == n.stats.sampling_rate == \
            e.stats.sampling_rate
    except AssertionError:
        msg = ('AR picker needs same sampling rate on all traces '
               'but provided stream was:\n%s') % st
        print(msg)
    spr = z.stats.sampling_rate
    p, s = ar_pick(z.data, n.data, e.data, spr, f1, f2, lta_p, sta_p,
                   lta_s, sta_s, m_p, m_s, l_p, l_s)
    picks = []
    #we write the phases as obspy pick struct for possible futher use in location codes
    for t, phase_hint, tr in zip((p, s), 'PS', (z, n)):
        if t > 0:
            pick = Pick(time=z.stats.starttime.timestamp+t,
                                     waveform_id=WaveformStreamID(network_code=z.stats.network,
                                     station_code=z.stats.station),
                                     phase_hint=phase_hint)
            if phase_hint == "S" and (t - (picks[0].time.timestamp - z.stats.starttime.timestamp)) > 0 \
                               and (t - (picks[0].time.timestamp - z.stats.starttime.timestamp)) < 0.5:
                picks.append(pick)
            elif phase_hint == "P":
                picks.append(pick)
    return (picks)

Same settings and data sets as before

In [ ]:
par = AttribDict()
par.sds_rootdir = SDS_DATA_PATH

par.filter = AttribDict(type="bandpass", freqmin=5.0, freqmax=30.0,
                            corners=2, zerophase=True)
trace_ids = {}

trace_ids = {"XG.BB01..HHZ": 1, "XG.ADR1..GLZ": 1,"XG.C16..GLZ": 1,"XG.C17..GLZ": 1,"XG.C29..DLZ": 1,
             "XG.C2A..DLZ": 1,"XG.C68..DLZ": 1,"XG.C67..DLZ": 1,"XG.C65..DLZ": 1,
             "XG.C6A..DLZ": 1,"XG.C69..DLZ": 1}
par.trace_ids = trace_ids

#Here we restrict our picking for events with more than X triggered stations
par.threshold_coin = 7

par.dir = "./XG_trigger" 
par.logfile = os.path.join(par.dir, "%s_log.txt" % "XG")
par.trigfile = os.path.join(par.dir, "%s_trigger.txt" % "XG")

Let's use the STA/LTA triggers as first guess and try to get better picks for seleceted waveforms

In [ ]:
!rm "%s/AR-*.png"%(par.dir)

In [ ]:
import numpy as np
trig_data = np.genfromtxt("%s/XG_trigger.txt"%par.dir,usecols=(1,2,3),dtype='str',delimiter=" ")


def getdata(t1):
    num_stations = 0
    station_id = []
    exceptions = []
    st= Stream()
    for trace_id,_ in par.trace_ids.items():
        net, sta, loc, cha = trace_id.split(".")
        station_id.append("%s.%s.%s.%s*"%(net,sta,loc,cha[:-1]))
        
        for comp in "ZNE":
            cha_ = cha[:-1] + comp
            try:
                tmp = sds_cl.get_waveforms(network=net, station=sta, location=loc, channel=cha_, starttime=t1, endtime=t1+3600)
                inv = read_inventory("./stationxml/station_%s_%s.xml"%(net,sta))
                tmp.attach_response(inv)
                st += tmp
                num_stations += 1
            except:
                exceptions.append("%s-%s" % (sta,cha_))
                continue
    if len(exceptions)>0:
        print(exceptions)
    st.merge(-1)
    st.sort()
    return st,station_id,num_stations

trigger_times = []
duration = []
coincidense = []
allpicks = []
sds_cl = sds.Client(sds_root=par.sds_rootdir)
t_0 = UTCDateTime(trig_data[0][0])
t_1 = UTCDateTime(t_0.year,t_0.month,t_0.day,t_0.hour) 

st,stations,no_s = getdata(t_1)

## We restrict ourself to 50 possible events ....

for i in range(0,50): #len(trig_data)):
    t_t = (UTCDateTime(trig_data[i][0]))
    duration = float(trig_data[i][1])
    coin = float(trig_data[i][2])
    if coin > par.threshold_coin:
        if UTCDateTime(t_t.year,t_t.month,t_t.day,t_t.hour) > t_1:
            #get a new junk of data
            st,stations,no_s = getdata(UTCDateTime(t_t.year,t_t.month,t_t.day,t_t.hour))

        #prepare plotting as well
        dpi = 72
        fig = plt.figure(figsize=(700.0 / dpi, 400.0 / dpi))
        ax = None
        stt = st.copy()
        stt.detrend("linear")
        
        stt.filter(**par.filter)
        stt.trim(t_t-1,t_t+duration+1)
        stt.taper(0.01,type="cosine")

        for i_,trace_id in enumerate(stations):
            net, sta, loc, cha = trace_id.split(".")
            sstt = stt.select(station=sta)
            ax = fig.add_subplot(len(stations), 1, i_ + 1, sharex=ax)
            try:
                picks = _arpicker(sstt)
            except:
                print("did not work for station: ",sta)
            stat = {}
            for p in picks:
                if p.phase_hint == 'P':
                    stat.update({"network": net})
                    stat.update({"station": sta})
                    stat.update({"P": p.time})
                    stat.update({"channel": cha[:-1]+"Z"})
                if p.phase_hint == 'S':
                    stat.update({"S": p.time})
            if stat["P"] or stat["s"]:
                tr = sstt.select(station=sta,component="Z")[0]
                x = tr.times(type="matplotlib") 
                x = tr.times(type="matplotlib") - tr.stats.starttime.matplotlib_date
                ax.plot(x, tr.data, color="k", linewidth=1.2)
                ax.text(0.02, 0.95, sta, va="top", ha="left",
                        transform=ax.transAxes) 
                try:
                    ax.axvline(x=stat["P"].matplotlib_date - tr.stats.starttime.matplotlib_date,color="r")
                except:
                    pass
                try:
                    ax.axvline(x=stat["S"].matplotlib_date- tr.stats.starttime.matplotlib_date,color="b")
                except:
                    pass
        
        fig.subplots_adjust(hspace=0)            
        plt.savefig('%s/AR-%04d%02d%02dT%02d-%d.png'%(par.dir,t_t.year,t_t.month,t_t.day,t_t.hour,i),dpi=300)
        plt.close('all')
            
        